In [ ]:
import numpy as np
import struct

In [ ]:
def load_mnist_images(filename):
    with open(filename, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        print(num, rows, cols)
        images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows*cols)
        return images / 255.0

def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

In [ ]:
X_train = load_mnist_images('datasets/MNIST/train-images.idx3-ubyte')
y_train = load_mnist_labels('datasets/MNIST/train-labels.idx1-ubyte')
X_test = load_mnist_images('datasets/MNIST/t10k-images.idx3-ubyte')
y_test = load_mnist_labels('datasets/MNIST/t10k-labels.idx1-ubyte')

In [ ]:
np.random.seed(42)

In [ ]:
input_size = 784 # 28 x 28 pixels
hidden_size1 = 128
hidden_size2 = 64
output_size = 10 # numbers from 0 to 9

In [ ]:
weights = {
    "W1": np.random.randn(input_size, hidden_size1) * np.sqrt(2.0 / input_size),
    "W2": np.random.randn(hidden_size1, hidden_size2) * np.sqrt(2.0 / hidden_size1),
    "W3": np.random.randn(hidden_size2, output_size) * np.sqrt(2.0 / hidden_size2),
    "b1": np.zeros((1, hidden_size1)),
    "b2": np.zeros((1, hidden_size2)),
    "b3": np.zeros((1, output_size))
}

In [ ]:
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True)) # numerical stability (removal of very large nums as exponentiation will result in large nums)
    return expZ / np.sum(expZ, axis=1, keepdims=True) # normalization

In [ ]:
def forward_propagation(X, weights):
    Z1 = np.dot(X, weights["W1"]) + weights["b1"]
    A1 = relu(Z1)

    Z2 = np.dot(A1, weights["W2"]) + weights["b2"]
    A2 = relu(Z2)

    Z3 = np.dot(A2, weights["W3"]) + weights["b3"]
    A3 = softmax(Z3)

    return Z1, A1, Z2, A2, Z3, A3  

In [ ]:
def compute_loss(Y_pred, Y_true, weights, lambda_):
    m = Y_true.shape[0]
    log_likelihood = -np.log(Y_pred[range(m), Y_true])
    loss = np.sum(log_likelihood) / m

    L2_regularizer = (lambda_ / (2 * m) * (
        np.sum(weights["W1"] ** 2) + np.sum(weights["W2"] ** 2) + np.sum(weights["W3"] ** 2)
    ))

    return loss + L2_regularizer

In [ ]:
def backpropagation(X, Y_true, A1, A2, A3, weights, learning_rate, lambda_):
    m = X.shape[0]

    Y_one_hot = np.zeros((m, output_size))
    Y_one_hot[np.arange(m), Y_true] = 1

    dZ3 = A3 - Y_one_hot
    dW3 = (np.dot(A2.T, dZ3) + lambda_ * weights["W3"]) / m
    db3 = np.sum(dZ3, axis=0, keepdims=True) / m

    dA2 = np.dot(dZ3, weights["W3"].T)
    dZ2 = dA2 * (A2 > 0)  # ReLU derivative
    dW2 = (np.dot(A1.T, dZ2) + lambda_ * weights["W2"]) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = np.dot(dZ2, weights["W2"].T)
    dZ1 = dA1 * (A1 > 0)  # ReLU derivative
    dW1 = (np.dot(X.T, dZ1) + lambda_ * weights["W1"]) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    # Update weights
    weights["W1"] -= learning_rate * dW1
    weights["b1"] -= learning_rate * db1
    weights["W2"] -= learning_rate * dW2
    weights["b2"] -= learning_rate * db2
    weights["W3"] -= learning_rate * dW3
    weights["b3"] -= learning_rate * db3

In [ ]:
epochs = 50
batch_size = 64
lambda_ = 0.01 # L2 regularization factor

In [ ]:
for epoch in range(epochs):
    shuffle_indices = np.random.permutation(X_train.shape[0])
    X_train, y_train = X_train[shuffle_indices], y_train[shuffle_indices]

    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]

        Z1, A1, Z2, A2, Z3, A3 = forward_propagation(X_batch, weights)

        learning_rate = 0.01 / (1 + 0.01 * epoch) # Learning rate decay
        backpropagation(X_batch, y_batch, A1, A2, A3, weights, learning_rate, lambda_)


    _, _, _, _, _, train_pred = forward_propagation(X_train, weights)
    train_loss = compute_loss(train_pred, y_train, weights, lambda_)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}")
    

In [ ]:
def predict(X, weights):
    _, _, _, _, _, A3 = forward_propagation(X, weights)
    return np.argmax(A3, axis=1)

y_pred = predict(X_test, weights)
accuracy = np.mean(y_pred == y_test) * 100
print(f"Test Accuracy: {accuracy:.2f}%")